In [2]:
import os
import cv2
from keras.preprocessing import image
import matplotlib.pyplot as plt 
import numpy as np
from keras.utils.np_utils import to_categorical
import random,shutil
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model
import json


In [3]:
def generator(dir, gen=image.ImageDataGenerator(rescale=1./255), shuffle=True,batch_size=1,target_size=(24,24),class_mode='categorical' ):

    return gen.flow_from_directory(dir,batch_size=batch_size,shuffle=shuffle,color_mode='grayscale',class_mode=class_mode,target_size=target_size)


In [4]:
TS=(24,24)
train_batch= generator('C:\\DISSERTATION ICBT\\final project\\Drowsiness_Detection_Application_CNN\\Python App\\data\\train',shuffle=True, batch_size=320,target_size=TS)
valid_batch = generator('C:\\DISSERTATION ICBT\\final project\\Drowsiness_Detection_Application_CNN\\Python App\\data\\valid',shuffle=True, batch_size=40,target_size=TS)
test_batch= generator('C:\\DISSERTATION ICBT\\final project\\Drowsiness_Detection_Application_CNN\\Python App\\data\\test',shuffle=True, batch_size=10,target_size=TS)
Train = len(train_batch.classes)//320
Valid = len(valid_batch.classes)//40
Test = len(test_batch.classes)//10
print(Train,Valid,Test)

Found 3200 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 30 images belonging to 2 classes.
10 20 3


In [5]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
#64 convolution filters used each of size 3x3
#choose the best features via pooling
#randomly turn neurons on and off to improve convergence
    Dropout(0.25),
#flatten since too many dimensions, we only want a classification output
    Flatten(),
#fully connected to get all relevant data
    Dense(128, activation='relu'),
#one more dropout for convergence' sake :) 
    Dropout(0.5),
#output a softmax to squash the matrix into output probabilities
    Dense(2, activation='softmax')
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 22, 22, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 20, 20, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 18, 18, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 18, 18, 64)       0

In [7]:
#The Adam optimization algorithm is an extension to stochastic gradient descent, loss function is for calculating loss
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
#our model will train on train_batches and will validate on valid_batches, We have made 20 epochs.
#Verbose is 2, which just specifies the verbosity of the log output printed to the console during training.
#We have 3200 training images and 320 per batch so steps_per_epoch is 10
#we have 800 validation images and 40 per batch so validation_steps is 20

history = model.fit_generator(train_batch, validation_data=valid_batch,epochs=20,steps_per_epoch=10 ,validation_steps=20,verbose=2)

C:\Users\Lavan\AppData\Local\Temp\ipykernel_25248\3630582741.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_batch, validation_data=valid_batch,epochs=20,steps_per_epoch=10 ,validation_steps=20,verbose=2)


Epoch 1/20
10/10 - 6s - loss: 0.6166 - accuracy: 0.7134 - val_loss: 0.4308 - val_accuracy: 0.8300 - 6s/epoch - 623ms/step
Epoch 2/20
10/10 - 3s - loss: 0.3045 - accuracy: 0.8662 - val_loss: 0.1981 - val_accuracy: 0.9025 - 3s/epoch - 347ms/step
Epoch 3/20
10/10 - 4s - loss: 0.1782 - accuracy: 0.9262 - val_loss: 0.1253 - val_accuracy: 0.9538 - 4s/epoch - 417ms/step
Epoch 4/20
10/10 - 4s - loss: 0.1176 - accuracy: 0.9566 - val_loss: 0.1194 - val_accuracy: 0.9438 - 4s/epoch - 395ms/step
Epoch 5/20
10/10 - 4s - loss: 0.0843 - accuracy: 0.9731 - val_loss: 0.0713 - val_accuracy: 0.9775 - 4s/epoch - 395ms/step
Epoch 6/20
10/10 - 4s - loss: 0.0561 - accuracy: 0.9853 - val_loss: 0.0427 - val_accuracy: 0.9875 - 4s/epoch - 389ms/step
Epoch 7/20
10/10 - 4s - loss: 0.0336 - accuracy: 0.9906 - val_loss: 0.0298 - val_accuracy: 0.9875 - 4s/epoch - 383ms/step
Epoch 8/20
10/10 - 4s - loss: 0.0309 - accuracy: 0.9916 - val_loss: 0.0341 - val_accuracy: 0.9900 - 4s/epoch - 380ms/step
Epoch 9/20
10/10 - 4s - 

In [ ]:
plt.figure(1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['Training', 'Validation'])
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.figure(2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training', 'Validation'])
plt.title('Loss')
plt.xlabel('Epochs')
plt.show()
     

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

In [ ]:
plt.plot(epochs, accuracy, "b", label="Training accuracy")
plt.plot(epochs, val_accuracy, "r", label="Testing accuracy")
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, loss, "b", label="Training loss")
plt.plot(epochs, val_loss, "r", label="Testing loss")
plt.legend()
plt.show()

In [9]:
model.save('model/trained.h5', overwrite=True) #Saving our model

In [10]:
# Convert the Keras model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('model/trained.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Lavan\AppData\Local\Temp\tmpdxkca681\assets


INFO:tensorflow:Assets written to: C:\Users\Lavan\AppData\Local\Temp\tmpdxkca681\assets


In [11]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('model/trained.tflite', 'wb') as f:
    f.write(tflite_model)

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path='model/trained.tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Create metadata dictionary
metadata = {
    "input": {
        "name": input_details[0]['name'],
        "shape": input_details[0]['shape'].tolist(),
        "dtype": str(input_details[0]['dtype'])
    },
    "output": {
        "name": output_details[0]['name'],
        "shape": output_details[0]['shape'].tolist(),
        "dtype": str(output_details[0]['dtype'])
    }
}

# Save metadata as JSON
with open('model/trained.json', 'w') as f:
    json.dump(metadata, f)


INFO:tensorflow:Assets written to: C:\Users\Lavan\AppData\Local\Temp\tmpnyn7hfj_\assets


INFO:tensorflow:Assets written to: C:\Users\Lavan\AppData\Local\Temp\tmpnyn7hfj_\assets
